In [1]:
import awkward as ak
import hist
import numpy as np
from coffea import util as cutil
from coffea.lookup_tools.dense_lookup import dense_lookup
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

btagWPs = {
    "deepJet": {
        "2016preVFP_UL": {
            "L": 0.0508,
            "M": 0.2598,
            "T": 0.6502,
        },
        "2016postVFP_UL": {
            "L": 0.0480,
            "M": 0.2489,
            "T": 0.6377,
        },
        "2017_UL": {
            "L": 0.0532,
            "M": 0.3040,
            "T": 0.7476,
        },
        "2018_UL": {
            "L": 0.0490,
            "M": 0.2783,
            "T": 0.7100,
        },
    },
    "deepCSV": {
        "2016preVFP_UL": {
            "L": 0.2027,
            "M": 0.6001,
            "T": 0.8819,
        },
        "2016postVFP_UL": {
            "L": 0.1918,
            "M": 0.5847,
            "T": 0.8767,
        },
        "2017_UL": {
            "L": 0.1355,
            "M": 0.4506,
            "T": 0.7738,
        },
        "2018_UL": {
            "L": 0.1208,
            "M": 0.4168,
            "T": 0.7665,
        },
    },
}

# single TT files to derive efficiency

taggerBranch = {"deepJet": "btagDeepFlavB", "deepCSV": "btagDeepB"}


file1 = "/uscms_data/d3/jieun201/YOURWORKINGAREA/nano_mc2016pre_3-146.root"

events = NanoEventsFactory.from_root(
    file1,
entry_stop=100_000, schemaclass=NanoAODSchema,
).events()
year = '2016preVFP_UL'

#events = file1.events()

# b-tagging only applied for jets with |eta| < 2.5
phasespace_cuts = (events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.5)
jets = ak.flatten(events.Jet[phasespace_cuts])

bins = [40,60,80,100,120,140,160,180,200,220,240,260,280,300,1000]


#for tag in ["deepJet", "deepCSV"]:
for tag in ["deepJet"]:
    #for wp in ["L", "M", "T"]:
    for wp in ["M"]:
        efficiencyinfo = (
            hist.Hist.new.Variable(bins, name="pt")
            .Reg(4, 0, 2.5, name="abseta")
            .IntCat([0, 4, 5], name="flavor")
            .Bool(name="passWP")
            .Double()
            .fill(
                pt=jets.pt,
                abseta=abs(jets.eta),
                flavor=jets.hadronFlavour,
                passWP=jets[taggerBranch[tag]] > btagWPs[tag][year][wp],
            )
        )
        eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
        efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
        #print(tag, wp, efflookup(np.array([42, 60]), 0.2, 2))

    #    cutil.save(efflookup, f"btageff_{tag}_{wp}_{year}.coffea")

/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1813 branches) at 0x7f25388fe170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1813 branches) at 0x7f25388fe170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1813 branches) at 0x7f25388fe170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1813 branches) at 0x7f25388fe170>, taking first instance
  warnings.warn(
/home/jieun201/.loca

In [2]:
print(eff)

Hist(
  Variable([40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 1000], name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: 53.253257583092044 (nan with flow)


In [3]:
#s_2d = eff.project("pt","abseta")
#s_2d[{"flavor": 4j}]

In [4]:
eff_4=eff[{"flavor": 4j}]

In [5]:
eff_4.values()

array([[0.15014866, 0.1463964 , 0.14943457, 0.13351499],
       [0.12843407, 0.14563929, 0.15162907, 0.13404255],
       [0.15214994, 0.14663144, 0.14285714, 0.09621993],
       [0.13628319, 0.14132762, 0.13522013, 0.12048193],
       [0.1242236 , 0.11397059, 0.12820513, 0.13043478],
       [0.1146789 , 0.13068182, 0.16239316, 0.03571429],
       [0.10852713, 0.13978495, 0.16666667, 0.10344828],
       [0.13333333, 0.234375  , 0.15      , 0.        ],
       [0.3125    , 0.19047619, 0.22727273, 0.06666667],
       [0.23076923, 0.11111111, 0.07692308, 0.3       ],
       [0.125     , 0.05      , 0.2       , 0.125     ],
       [0.19230769, 0.11111111, 0.22222222, 0.5       ],
       [0.11111111, 0.25      , 0.25      , 0.5       ],
       [0.23287671, 0.2       , 0.1875    , 0.25      ]])

In [6]:
eff_5=eff[{"flavor": 5j}]

In [7]:
eff_5.values()

array([[0.78410903, 0.7682761 , 0.70583851, 0.63814691],
       [0.80113435, 0.77900779, 0.72297715, 0.66685457],
       [0.81170158, 0.78869407, 0.74451902, 0.6525497 ],
       [0.8079096 , 0.80499076, 0.7414421 , 0.69387755],
       [0.81218905, 0.81656805, 0.78064516, 0.6875    ],
       [0.8246493 , 0.81282051, 0.80272109, 0.72596154],
       [0.84285714, 0.83529412, 0.73103448, 0.73770492],
       [0.84223919, 0.828125  , 0.74834437, 0.55882353],
       [0.84331797, 0.79234973, 0.85964912, 0.66      ],
       [0.84795322, 0.77099237, 0.82608696, 0.47826087],
       [0.83035714, 0.74509804, 0.67391304, 0.75      ],
       [0.82894737, 0.90740741, 0.81481481, 0.625     ],
       [0.77777778, 0.77142857, 0.76190476, 1.        ],
       [0.795     , 0.80985915, 0.65789474, 0.76923077]])

In [8]:
eff_0=eff[{"flavor": 0j}]

In [9]:
eff_0.values()

array([[0.01304503, 0.01384012, 0.01565052, 0.02020417],
       [0.00907229, 0.01380572, 0.01528633, 0.01375465],
       [0.00880743, 0.01439247, 0.00818076, 0.01154107],
       [0.00867769, 0.00718047, 0.01210962, 0.01605505],
       [0.01144011, 0.01052632, 0.01607717, 0.01538462],
       [0.00785634, 0.01331719, 0.019678  , 0.01759531],
       [0.00822368, 0.0125261 , 0.00808625, 0.03030303],
       [0.00791557, 0.01290323, 0.00819672, 0.02173913],
       [0.00671141, 0.01276596, 0.01829268, 0.03191489],
       [0.01675978, 0.01282051, 0.02362205, 0.        ],
       [0.00735294, 0.00952381, 0.06185567, 0.02380952],
       [0.        , 0.        , 0.03076923, 0.02941176],
       [0.        , 0.02325581, 0.04347826, 0.08695652],
       [0.03296703, 0.03422053, 0.11229947, 0.10465116]])